# Responsi NLP
Muhammad Ihsan Prawira Hutomo
2211110022

Pada responsi QAS dengan daset mentalhealth saya mencoba 2 pendekatan yaitu menggunakan TF-IDF dengan Cosine Similarity dan IndoBERT QA pretrained model


# TF-IDF with Cosine Similarity

## Import Data & Installations


In [4]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


In [5]:
df = pd.read_csv('/content/dataset_mentalhealth.csv')


In [2]:
!pip install nltk

In [3]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## 1. Text Preprocessing

Tidak banyak yang dilakukan pada preprocessing pada dataset ini karena data telah berbentuk bersih, terstsruktur, dan tidak terdapat gangguan lainnya seperti whitespaces, link yang tidak jelas, digit tidak ada arti, kata-kata yang disingkat, dll.

In [6]:
# Fungsi untuk membersihkan teks
def clean_text(text):
    # Menghapus karakter non-alfanumerik dan mengonversi teks menjadi huruf kecil
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text).lower()
    return text

# Fungsi untuk tokenisasi dan penghapusan stop words
def remove_stopwords(text):
    stop_words = set(stopwords.words('indonesian'))
    words = text.split()
    words = [word for word in words if word.isalnum() and word not in stop_words]
    return ' '.join(words)

# Membersihkan dan tokenisasi pertanyaan
df['clean_questions'] = df['Questions'].apply(clean_text)
df['clean_questions'] = df['clean_questions'].apply(remove_stopwords)

# Membersihkan dan tokenisasi jawaban
df['clean_answers'] = df['Jawaban'].apply(clean_text)
df['clean_answers'] = df['clean_answers'].apply(remove_stopwords)


In [7]:
df

,Question_ID,Questions,Jawaban,clean_questions,clean_answers
0,1590140,Apa yang dimaksud dengan penyakit mental?,Penyakit mental adalah kondisi kesehatan yang ...,penyakit mental,penyakit mental kondisi kesehatan mengganggu p...
1,2110618,Siapa yang terpengaruh oleh penyakit mental?,Diperkirakan bahwa penyakit mental mempengaruh...,terpengaruh penyakit mental,penyakit mental mempengaruhi 1 5 orang dewasa ...
2,6361820,Apa penyebab penyakit mental?,Diperkirakan bahwa penyakit mental mempengaruh...,penyebab penyakit mental,penyakit mental mempengaruhi 1 5 orang dewasa ...
3,9434130,Apa sajakah tanda-tanda peringatan penyakit me...,Gejala gangguan kesehatan mental bervariasi te...,sajakah tandatanda peringatan penyakit mental,gejala gangguan kesehatan mental bervariasi te...
4,7657263,Apakah penderita penyakit jiwa bisa sembuh?,"Ketika penyembuhan dari penyakit mental, ident...",penderita penyakit jiwa sembuh,penyembuhan penyakit mental identifikasi pengo...
...,...,...,...,...,...
93,4373204,Bagaimana saya tahu kalau saya minum terlalu b...,Menyortir jika Anda minum terlalu banyak bisa ...,minum,menyortir minum rumit unik hubungan alkohol un...
94,7807643,"Jika ganja berbahaya, mengapa kita melegalkannya?","Asap ganja, misalnya, mengandung racun penyeba...",ganja berbahaya melegalkannya,asap ganja mengandung racun penyebab kanker ri...
95,4352464,Bagaimana cara meyakinkan anak saya untuk tida...,Anda tidak bisa. Tetapi Anda dapat mempengaruh...,anak narkoba,mempengaruhi kapasitas pilihan dunia alkohol o...
96,6521784,Apa status hukum (dan bukti) minyak CBD?,Cannabidiol atau CBD adalah komponen ganja yan...,status hukum bukti minyak cbd,cannabidiol cbd komponen ganja alami diekstrak...


In [8]:
# Menggabungkan teks dari kolom pertanyaan dan jawaban
df['combined_text'] = df['clean_questions'] + ' ' + df['clean_answers']


## 2. Vektoriasi: TF-IDF Process

TF IDF Menggunakan hasil dari penggabungan teks dari 'clean_questions' dan 'clean_answers' sehingga model diharapkan akan membuat vektorisasi berdasarkan kata yang terdapat di dua kolom tersebut.

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Membuat vektor TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['combined_text'])


In [10]:
tfidf_matrix

<98x2107 sparse matrix of type '<class 'numpy.float64'>'
	with 7296 stored elements in Compressed Sparse Row format>

## 3. Modelling :Cosine Similarity

Metode Cosine Similarity adalah mengukur kemiripan antara
dua dokumen atau teks. Pada Cosine Similarity dokumen atau
teks dianggap sebagai vector. Pada penelitian ini, Cosine
Similarity digunakan untuk menghitung jumlah kata istilah yang
muncul pada halaman-halaman yang diacu pada daftar indeks.
Semakin banyak jumlah kata istilah yang muncul pada suatu
halaman semakin tinggi nilai Cosine Similarity yang diperoleh.

Sehingga nanti pada pemodelan cosine similarity dihitung berdasarkan question_vector dengan tfidf_matrix yang telah dilakukan.

In [27]:
from sklearn.metrics.pairwise import cosine_similarity

# Fungsi untuk mendapatkan jawaban berdasarkan pertanyaan menggunakan cosine similarity
def get_answer(question, tfidf_matrix):
    # Menghasilkan vektor TF-IDF untuk pertanyaan
    question_vector = tfidf_vectorizer.transform([question])

    # Menghitung cosine similarity antara vektor pertanyaan dan vektor jawaban
    similarities = cosine_similarity(question_vector, tfidf_matrix).flatten()

    # Mendapatkan indeks jawaban dengan similarity tertinggi
    answer_index = similarities.argmax()

    # Mengembalikan jawaban dari indeks yang ditemukan
    return df['Jawaban'][answer_index]

# Contoh penggunaan
sample_question = "Penyakit mental yang delusi"
answer = get_answer(sample_question, tfidf_matrix)
print("Jawaban:", answer)


Jawaban: 'Kesehatan mental' dan 'penyakit mental' semakin banyak digunakan seolah -olah memaksudkan hal yang sama, tetapi tidak. Setiap orang memiliki kesehatan mental, sama seperti semua orang memiliki kesehatan. Seperti yang dikatakan Organisasi Kesehatan Dunia dengan terkenal, “Tidak ada kesehatan tanpa kesehatan mental.” Dalam perjalanan seumur hidup, tidak semua orang akan mengalami penyakit mental, tetapi semua orang akan berjuang atau memiliki tantangan dengan kesejahteraan mental mereka (mis., Kesehatan mental mereka) sama seperti kita semua memiliki tantangan dengan kesejahteraan fisik kita dari waktu demi waktu.
 Ketika kita berbicara tentang kesehatan mental, kita berbicara tentang kesejahteraan mental kita: emosi kita, pikiran dan perasaan kita, kemampuan kita untuk menyelesaikan masalah dan mengatasi kesulitan, koneksi sosial kita, dan pemahaman kita tentang dunia di sekitar kita.
 Penyakit mental adalah penyakit yang memengaruhi cara orang berpikir, merasakan, berperilaku

## Contoh Implementasi

Diberikan 10 teks pengujian dengan 5 adalah pertanyaan relevan(belum tentu terdapat jawaban di dalam dataset) dan 5 pertanyaan irrelevant

In [12]:
import pandas as pd

# Buat pertanyaan-pertanyaan uji
test_questions = [
    "Apakah terapi kognitif efektif untuk mengatasi stres?",
    "Apa itu ADHD dan bagaimana cara diagnosisnya?",
    "Apakah makanan tertentu dapat memengaruhi kesehatan mental?",
    "Bagaimana cara mengenali tanda-tanda psikosis?",
    "Apakah olahraga dapat membantu meningkatkan kesehatan mental?",
    "Berapa banyak planet di tata surya kita?",
    "Bagaimana cara merawat tanaman hias dalam ruangan?",
    "Siapa presiden pertama Amerika Serikat?",
    "Apa yang dimaksud dengan hukum permintaan dan penawaran?",
    "Bagaimana cara mengatasi rambut bercabang?"]

# Fungsi untuk melakukan uji dan analisis hasilnya
def perform_test(questions, tfidf_matrix):
    results = []

    for idx, test_question in enumerate(questions):
        predicted_answer = get_answer(test_question, tfidf_matrix)

        result_dict = {
            "Pertanyaan": test_question,
            "Jawaban Prediksi": predicted_answer
        }

        results.append(result_dict)

        print(f"Pertanyaan {idx + 1}: {test_question}")
        print(f"Jawaban Prediksi: {predicted_answer}\n")

    return pd.DataFrame(results)

# Simpan hasil uji ke Excel
results_df = perform_test(test_questions, tfidf_matrix)
results_df.to_excel("TF-IDF Cosine Similarity.xlsx.xlsx", index=False)


Pertanyaan 1: Apakah terapi kognitif efektif untuk mengatasi stres?
Jawaban Prediksi: Berbagai jenis terapi lebih efektif berdasarkan sifat kondisi kesehatan mental dan/atau gejala dan orang yang memilikinya (misalnya, anak -anak akan mendapat manfaat dari terapis yang berspesialisasi dalam kesehatan mental anak -anak). Namun, ada beberapa jenis perawatan dan terapi yang dapat membantu.

Pertanyaan 2: Apa itu ADHD dan bagaimana cara diagnosisnya?
Jawaban Prediksi: Sementara kita menganggap gangguan perhatian/hiperaktif sebagai penyakit yang mempengaruhi anak-anak, itu bisa bertahan hingga dewasa-dan beberapa orang tidak didiagnosis sampai di kemudian hari.
 ADHD didiagnosis seperti penyakit mental lainnya, jadi dokter keluarga Anda adalah tempat yang baik untuk memulai. Dokter keluarga Anda mungkin dapat menilai kesehatan mental Anda dan memulai perawatan, jika diperlukan, atau mereka dapat merujuk Anda ke layanan kesehatan mental yang lebih khusus.
 Ada beberapa organisasi yang dapat 

## 5. Evaluasi

- Model QAS hanya mencari jawaban dengan similarity paling tinggi saja, jika terdapat jawaban yang tidak relevan juga tetap memberikan jawaban sehingga jawaban yang dikeluarkan tidak masuk dengan pertanyaan.

Maka perlu diperlakukan treshold pada nilai similarity index atau answer_index yang mana disini nilai yang terbaik dari melihat percobaan yang telah dilakukan adalah 0.15 .


In [13]:
def get_answer(question, tfidf_matrix, threshold=0.15):
    # Menghasilkan vektor TF-IDF untuk pertanyaan
    question_vector = tfidf_vectorizer.transform([question])

    # Menghitung cosine similarity antara vektor pertanyaan dan vektor jawaban
    similarities = cosine_similarity(question_vector, tfidf_matrix).flatten()

    # Mendapatkan indeks jawaban dengan similarity tertinggi
    answer_index = similarities.argmax()

    # Jika similarity di bawah threshold, anggap pertanyaan kurang relevan
    if similarities[answer_index] < threshold:
        return "Pertanyaan kurang relevan..."

    # Mengembalikan jawaban dari indeks yang ditemukan
    return df['Jawaban'][answer_index]

In [14]:
# Buat pertanyaan-pertanyaan uji
test_questions = [
    "Apakah terapi kognitif efektif untuk mengatasi stres?",
    "Apa itu ADHD dan bagaimana cara diagnosisnya?",
    "Apakah makanan tertentu dapat memengaruhi kesehatan mental?",
    "Bagaimana cara mengenali tanda-tanda psikosis?",
    "Apakah olahraga dapat membantu meningkatkan kesehatan mental?",
    "Berapa banyak planet di tata surya kita?",
    "Bagaimana cara merawat tanaman hias dalam ruangan?",
    "Siapa presiden pertama Amerika Serikat?",
    "Apa yang dimaksud dengan hukum permintaan dan penawaran?",
    "Bagaimana cara mengatasi rambut bercabang?"]





In [16]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

def perform_test(questions, tfidf_matrix):
    results = []

    for idx, test_question in enumerate(questions):
        predicted_answer = get_answer(test_question, tfidf_matrix)

        # Get similarity index for the question
        question_vector = tfidf_vectorizer.transform([test_question])
        similarities = cosine_similarity(question_vector, tfidf_matrix).flatten()
        similarity_index = similarities.max()
        answer_index = similarities.argmax()

        result_dict = {
            "Pertanyaan": test_question,
            "Jawaban Prediksi": predicted_answer,
            "Similarity Index": similarity_index
        }

        results.append(result_dict)

        print(f"Pertanyaan {idx + 1}: {test_question}")
        print(f"Jawaban Prediksi: {predicted_answer}")
        print(f"Similarity Index: {similarity_index}")
        print()

    return pd.DataFrame(results)

# Perform the test and export the results to Excel
results_df = perform_test(test_questions, tfidf_matrix)
results_df.to_excel("TF-IDF Cosine Similarity dengan Score.xlsx", index=False)


Pertanyaan 1: Apakah terapi kognitif efektif untuk mengatasi stres?
Jawaban Prediksi: Berbagai jenis terapi lebih efektif berdasarkan sifat kondisi kesehatan mental dan/atau gejala dan orang yang memilikinya (misalnya, anak -anak akan mendapat manfaat dari terapis yang berspesialisasi dalam kesehatan mental anak -anak). Namun, ada beberapa jenis perawatan dan terapi yang dapat membantu.
Similarity Index: 0.3096413619139565

Pertanyaan 2: Apa itu ADHD dan bagaimana cara diagnosisnya?
Jawaban Prediksi: Sementara kita menganggap gangguan perhatian/hiperaktif sebagai penyakit yang mempengaruhi anak-anak, itu bisa bertahan hingga dewasa-dan beberapa orang tidak didiagnosis sampai di kemudian hari.
 ADHD didiagnosis seperti penyakit mental lainnya, jadi dokter keluarga Anda adalah tempat yang baik untuk memulai. Dokter keluarga Anda mungkin dapat menilai kesehatan mental Anda dan memulai perawatan, jika diperlukan, atau mereka dapat merujuk Anda ke layanan kesehatan mental yang lebih khusus.

## Kesimpulan

Pada Metode ini Cosine Similarity hanya mencocokkan index kemiripan dengan jawaban yang ada, yang mana akan memberikan 1 baris penuh jawaban jika pertanyaan terdapat indikasi kemiripan

# IndoBERT

In [17]:
df = pd.read_csv('/content/dataset_mentalhealth.csv')

In [18]:
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("indolem/indobert-base-uncased")

# Example sentence
sentence = "Ini contoh kalimat untuk di-tokenize."

# Tokenize the sentence
tokens = tokenizer(sentence, return_tensors="pt")

print("Original Sentence:", sentence)
print("Tokenized Output:", tokens)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/234k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Original Sentence: Ini contoh kalimat untuk di-tokenize.
Tokenized Output: {'input_ids': tensor([[   3, 1540, 3081, 5859, 1559, 1485,   17, 2487, 6950, 7709,   18,    4]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [19]:
import re
import string
import torch
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity

class QASystem:
    def __init__(self, df):
        self.df = df
        self.tokenizer = AutoTokenizer.from_pretrained("indolem/indobert-base-uncased")
        self.model = AutoModel.from_pretrained("indolem/indobert-base-uncased")

    def clean_text(self, text):
        text = text.translate(str.maketrans('', '', string.punctuation))
        text = re.sub(r'\s+', ' ', text).strip()
        return text.lower()

    def vectorize_text(self, text):
        inputs = self.tokenizer(
            text,
            return_tensors="pt",
            max_length=512,
            truncation=True
        )
        with torch.no_grad():
            outputs = self.model(**inputs)
        return outputs['pooler_output'].numpy()

    def preprocess_dataframe(self):
        self.df['cleaned_Jawaban'] = self.df['Jawaban'].apply(self.clean_text)
        self.df['Vectorized_Jawaban'] = self.df['cleaned_Jawaban'].apply(self.vectorize_text)

    def vectorize_user_question(self, user_question):
        user_question = self.clean_text(user_question)
        user_question_embedding = self.vectorize_text(user_question)
        return user_question_embedding

    def answer_question(self, user_question):
        user_question_embedding = self.vectorize_user_question(user_question)

        # Calculate cosine similarity with all questions in the dataset
        similarities = cosine_similarity(
            user_question_embedding, np.vstack(self.df['Vectorized_Jawaban'])
        )

        # Find the index of the most similar question
        most_similar_index = np.argmax(similarities)
        similarity_score = similarities[0, most_similar_index]

        if similarity_score < 0.15:
            return "Pertanyaan kurang relevan...", similarity_score

        return self.df.loc[most_similar_index, 'Jawaban'], similarity_score

    def answer_user_input(self, user_input):
        answer_result, similarity_score = self.answer_question(user_input)
        print(f"Pertanyaan: {user_input}")
        print(f"Jawaban Prediksi: {answer_result}")
        print(f"Similarity Index: {similarity_score}")
        print("")

# Assuming df is your DataFrame with 'Questions' and 'Jawaban' columns
qas_model = QASystem(df)
qas_model.preprocess_dataframe()




pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

In [20]:
# Example user questions
test_questions = [
    "Apa gejala umum dari depresi?",
    "Bagaimana cara mendukung seseorang dengan gangguan kecemasan?",
    "Apakah terapi kognitif efektif untuk mengatasi stres?",
    "Apa itu ADHD dan bagaimana cara diagnosisnya?",
    "Bagaimana mengatasi perasaan stres di tempat kerja?",
    "Apakah makanan tertentu dapat memengaruhi kesehatan mental?",
    "Apa saja metode pengobatan yang umum digunakan untuk skizofrenia?",
    "Apa yang dimaksud dengan self-care dalam konteks kesehatan mental?",
    "Bagaimana cara mengenali tanda-tanda gangguan makan?",
    "Apakah olahraga dapat membantu meningkatkan kesehatan mental?",
    "Berapa banyak planet di tata surya kita?",
    "Apa resep masakan untuk membuat kue cokelat?",
    "Bagaimana cara merawat tanaman hias dalam ruangan?",
    "Apa saja jenis kamera yang direkomendasikan untuk fotografi profesional?",
    "Siapa presiden pertama Amerika Serikat?",
    "Apa itu teori relativitas Einstein?",
    "Bagaimana cara membuat aplikasi seluler?",
    "Apa yang dimaksud dengan hukum permintaan dan penawaran?",
    "Apa yang harus dikenakan pada pesta koktail?",
    "Bagaimana cara mengatasi rambut bercabang?"
]

# List to store results
user_results = []

# Answer user questions and store results
for user_question in test_questions:
    result = qas_model.answer_user_input(user_question)

    if result is not None:
        answer_result, similarity_score = result
        user_result = {
            "Pertanyaan Pengguna": user_question,
            "Jawaban": answer_result,
            "Similarity Index": similarity_score
        }
        user_results.append(user_result)
    else:
        print(f"Unable to answer question: {user_question}")

# Create DataFrame and export to Excel
user_results_df = pd.DataFrame(user_results)
user_results_df.to_excel("user_results.xlsx", index=False)

Pertanyaan: Apa gejala umum dari depresi?
Jawaban Prediksi: Jika Anda atau seseorang yang Anda kenal dalam krisis, perawatan rawat inap dapat membantu. Perawatan rawat inap dapat membantu orang menstabilkan obat baru, menyesuaikan dengan gejala baru, atau mendapatkan bantuan yang mereka butuhkan.
Similarity Index: 0.6018623113632202

Unable to answer question: Apa gejala umum dari depresi?
Pertanyaan: Bagaimana cara mendukung seseorang dengan gangguan kecemasan?
Jawaban Prediksi: Kita semua memiliki kesehatan mental yang terdiri dari keyakinan, pikiran, perasaan, dan perilaku kita.
Similarity Index: 0.6003204584121704

Unable to answer question: Bagaimana cara mendukung seseorang dengan gangguan kecemasan?
Pertanyaan: Apakah terapi kognitif efektif untuk mengatasi stres?
Jawaban Prediksi: Merasa nyaman dengan profesional yang Anda atau anak Anda bekerja sama sangat penting untuk keberhasilan perawatan. Menemukan profesional yang paling sesuai dengan kebutuhan Anda mungkin memerlukan pe

# IndoBert QA

## 1. Text Preprocessing

Pada teks preprocessing tidak dilakukan sama sekali pada model IndoBERT ini.

## 2. Vektorization & Modelling


Vektorisasi & Modelling dilakukan langsung dari pretrained model.

Bidirectional Training:
Model Indo-BERT QA menggunakan teknik pelatihan Transformer yang bersifat bidireksional. Pada pelatihan sebelumnya, model-model NLP cenderung menggunakan arah sekuensial, baik dari kiri ke kanan atau kombinasi dari kiri-ke-kanan dan kanan-ke-kiri. Indo-BERT mengadopsi pendekatan Transformer yang bidireksional, memungkinkan model memahami konteks dan alur bahasa dengan lebih mendalam.
Transformer dan Attention Mechanism:

Model ini menggunakan arsitektur Transformer dengan attention mechanism. Transformer terdiri dari dua mekanisme terpisah: encoder dan decoder. Namun, karena Indo-BERT bertujuan untuk menghasilkan model bahasa, hanya mekanisme encoder yang diperlukan. Transformer memanfaatkan attention mechanism untuk memahami relasi kontekstual antar kata atau sub-kata dalam teks.
Tokenisasi dan Vektorisasi:

Teks input, termasuk pertanyaan dan konteks jawaban, dipecah menjadi token menggunakan token tokenizer Indo-BERT. Token-token ini kemudian diubah menjadi vektor melalui mekanisme vektorisasi model. Representasi vektor ini mencerminkan pemahaman kontekstual dari setiap token berdasarkan seluruh konteks sekitarnya.

In [21]:
from transformers import pipeline
import pandas as pd

# Load your DataFrame
df = pd.read_csv('/content/dataset_mentalhealth.csv')

# Define the question-answering pipeline
qa_pipeline = pipeline(
    "question-answering",
    model="Rifky/Indobert-QA",
    tokenizer="Rifky/Indobert-QA"
)

def answer_question(row):
    question = row['Questions']
    context = row['Jawaban']
    inputs = {'question': question, 'context': context}
    answer = qa_pipeline(inputs)
    return answer['answer']

# Create a new column 'Answer' in your DataFrame
df['Answer'] = df.apply(answer_question, axis=1)

# Print the DataFrame with the new 'Answer' column
print(df[['Question_ID', 'Questions', 'Jawaban', 'Answer']])


config.json:   0%|          | 0.00/731 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/532 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/234k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/480k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

    Question_ID                                          Questions  \
0       1590140          Apa yang dimaksud dengan penyakit mental?   
1       2110618       Siapa yang terpengaruh oleh penyakit mental?   
2       6361820                      Apa penyebab penyakit mental?   
3       9434130  Apa sajakah tanda-tanda peringatan penyakit me...   
4       7657263        Apakah penderita penyakit jiwa bisa sembuh?   
..          ...                                                ...   
93      4373204  Bagaimana saya tahu kalau saya minum terlalu b...   
94      7807643  Jika ganja berbahaya, mengapa kita melegalkannya?   
95      4352464  Bagaimana cara meyakinkan anak saya untuk tida...   
96      6521784           Apa status hukum (dan bukti) minyak CBD?   
97      3221856                       Apa buktinya tentang vaping?   

                                              Jawaban  \
0   Penyakit mental adalah kondisi kesehatan yang ...   
1   Diperkirakan bahwa penyakit mental mempen

In [22]:
from transformers import pipeline
import pandas as pd

class InteractiveQAS:
    def __init__(self, model_name="Rifky/Indobert-QA"):
        self.qa_pipeline = pipeline("question-answering", model=model_name)
        self.df = pd.DataFrame()  # Initialize an empty DataFrame

    def load_data(self, data_frame):
        self.df = data_frame

    def answer_question(self, question):
        # Choose the 'Jawaban' column as the context
        context = " ".join(self.df['Jawaban'].tolist())

        # Use the QA pipeline to get the answer
        inputs = {'question': question, 'context': context}
        answer = self.qa_pipeline(inputs)

        return answer['answer']

# Example usage



In [14]:
df

,Question_ID,Questions,Jawaban,Answer
0,1590140,Apa yang dimaksud dengan penyakit mental?,Penyakit mental adalah kondisi kesehatan yang ...,adalah kondisi kesehatan yang mengganggu pikir...
1,2110618,Siapa yang terpengaruh oleh penyakit mental?,Diperkirakan bahwa penyakit mental mempengaruh...,1 dari 5 orang dewasa di Amerika
2,6361820,Apa penyebab penyakit mental?,Diperkirakan bahwa penyakit mental mempengaruh...,Ini dapat mempengaruhi siapa pun
3,9434130,Apa sajakah tanda-tanda peringatan penyakit me...,Gejala gangguan kesehatan mental bervariasi te...,ketika gejala berganda diekspresikan sekaligus
4,7657263,Apakah penderita penyakit jiwa bisa sembuh?,"Ketika penyembuhan dari penyakit mental, ident...",beberapa mungkin mengalami kembalinya gejala
...,...,...,...,...
93,4373204,Bagaimana saya tahu kalau saya minum terlalu b...,Menyortir jika Anda minum terlalu banyak bisa ...,Menyortir jika Anda minum terlalu banyak bisa ...
94,7807643,"Jika ganja berbahaya, mengapa kita melegalkannya?","Asap ganja, misalnya, mengandung racun penyeba...",memberikan kesempatan untuk menerapkan peratur...
95,4352464,Bagaimana cara meyakinkan anak saya untuk tida...,Anda tidak bisa. Tetapi Anda dapat mempengaruh...,"membantu mereka mengembangkan standar pribadi,..."
96,6521784,Apa status hukum (dan bukti) minyak CBD?,Cannabidiol atau CBD adalah komponen ganja yan...,legal


In [24]:
from transformers import pipeline
import pandas as pd

class InteractiveQAS:
    def __init__(self, model_name="Rifky/Indobert-QA"):
        self.qa_pipeline = pipeline("question-answering", model=model_name)
        self.df = pd.DataFrame()  # Initialize an empty DataFrame
        self.user_responses = []  # List to store user questions and answers

    def load_data(self, data_frame):
        self.df = data_frame

    def answer_question(self, question):
        # Choose the 'Jawaban' column as the context
        context = " ".join(self.df['Jawaban'].astype(str).tolist())

        # Use the QA pipeline to get the answer
        inputs = {'question': question, 'context': context}
        answer = self.qa_pipeline(inputs)

        return answer['answer']

    def get_user_responses(self):
        return self.user_responses

# Example usage
qas_system = InteractiveQAS(model_name="Rifky/Indobert-QA")

# Load your DataFrame with 'Questions' and 'Jawaban' columns
# df = ...

# Assuming you have loaded your DataFrame
qas_system.load_data(df)

# Define the list of predefined questions
test_questions = [
    "Apakah terapi kognitif efektif untuk mengatasi stres?",
    "Apa itu ADHD dan bagaimana cara diagnosisnya?",
    "Apakah makanan tertentu dapat memengaruhi kesehatan mental?",
    "Bagaimana cara mengenali tanda-tanda psikosis?",
    "Apakah olahraga dapat membantu meningkatkan kesehatan mental?",
    "Berapa banyak planet di tata surya kita?",
    "Bagaimana cara merawat tanaman hias dalam ruangan?",
    "Siapa presiden pertama Amerika Serikat?",
    "Apa yang dimaksud dengan hukum permintaan dan penawaran?",
    "Bagaimana cara mengatasi rambut bercabang?"]

# List to store results
user_responses = []

# Loop through each question and get the answer
for user_question in test_questions:
    # Get the answer
    answer = qas_system.answer_question(user_question)

    # Display the answer
    print(f"Question: {user_question}")
    print(f"Answer: {answer}")
    print("-" * 50)

    # Store the result
    user_responses.append({"Question": user_question, "Answer": answer})

# Create DataFrame and export to Excel
user_responses_df = pd.DataFrame(user_responses)
user_responses_df.to_excel("IndoBERT.xlsx", index=False)


Question: Apakah terapi kognitif efektif untuk mengatasi stres?
Answer: Perawatan ini sangat efektif
--------------------------------------------------
Question: Apa itu ADHD dan bagaimana cara diagnosisnya?
Answer: seperti penyakit mental lainnya
--------------------------------------------------
Question: Apakah makanan tertentu dapat memengaruhi kesehatan mental?
Answer: Kesehatan fisik dapat memiliki dampak besar pada kesehatan mental
--------------------------------------------------
Question: Bagaimana cara mengenali tanda-tanda psikosis?
Answer: DVD tips
--------------------------------------------------
Question: Apakah olahraga dapat membantu meningkatkan kesehatan mental?
Answer: berolahraga secara teratur
--------------------------------------------------
Question: Berapa banyak planet di tata surya kita?
Answer: 14
--------------------------------------------------
Question: Bagaimana cara merawat tanaman hias dalam ruangan?
Answer: mencuci tangan dan mengenakan topeng
----

Disini model BERT dapat memberikan jawaban bukan dalam bentuk 1 baris penuh, melainkan 1 potongan teks jawaban yang sesuai dengan pertanyaan. Disini terdapat kekurangan dimana jika dataset tidak lengkap maka jawaban yang dihasilkan juga tidak tepat. Namun jawaban tetap memberikan format jawaban sesuai yang ditanyakan, misal ditanya berapa model akan mencari data yang berbentuk angka dan jika ditanya siapa dia akan mencari entity yang terdapat pada dataset Jawaban

# Kesimpulan

Dari Dataset yang diberikan berjumlah 99 baris, saya mencoba 2 pendekatan yaitu TF-IDF Cosine Similarity dan IndoBERT
Pada model TF-IDF Cosine Similarity, model ini baik dalam memprediksi jawaban karena langsung memberikan 1 baris penuh jawaban yang mirip atau tingkat similarity indexnya tinggi dengan pertanyaan yang diberikan. Kekurangannya model ini akan selalu memberikan 1 konteks jawaban penuh dan terkadang jauh dari topik yang diberikan, seperti pada pertanyaan no 10.
Pada model IndoBERT QA model ini kurang baik karena memberikan jawaban dalam 1 potongan teks. Namun model dapat baik mencari format jawaban yang dimaksud lebih terperinci, jika ditanyan siapa dia akan mencari Entitas, jika ditanya berapa dia akan mencari jawaban berupa angka.
Untuk skor akurasi yaitu jumlah benar/total jawaban didapat yang dikurasi secara manual adalah sebagai berikut:
- TF-IDF  Cosine similarity = 90%
- IndoBERT QA = 40%

Maka, bisa dibilang model TF-IDF lebih baik dalam menjawab dengan memberikan 1 baris penuh jawaban